###Creating month wise partitioned parquet file from csv

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("order_date", DateType(), True),
    StructField("order_time", StringType(), True)])

orders = spark.read.format("csv").option("header",True).schema(schema).load("dbfs:/FileStore/orders.csv")
# orders.show()
orders_concat =orders.withColumn("Order_Hour",substring(orders.order_time,0,2).cast("Integer")).withColumn("Order_Month", substring(orders.order_date,6,2).cast("Integer"))
orders_concat.show()
orders_concat.write.format('parquet').partitionBy("Order_Month").mode("overwrite").save("dbfs:/FileStore/orders")



+--------+----------+----------+----------+-----------+
|order_id|order_date|order_time|Order_Hour|Order_Month|
+--------+----------+----------+----------+-----------+
|       1|2015-01-01|  11:38:36|        11|          1|
|       2|2015-01-01|  11:57:40|        11|          1|
|       3|2015-01-01|  12:12:28|        12|          1|
|       4|2015-01-01|  12:16:31|        12|          1|
|       5|2015-01-01|  12:21:30|        12|          1|
|       6|2015-01-01|  12:29:36|        12|          1|
|       7|2015-01-01|  12:50:37|        12|          1|
|       8|2015-01-01|  12:51:37|        12|          1|
|       9|2015-01-01|  12:52:01|        12|          1|
|      10|2015-01-01|  13:00:15|        13|          1|
|      11|2015-01-01|  13:02:59|        13|          1|
|      12|2015-01-01|  13:04:41|        13|          1|
|      13|2015-01-01|  13:11:55|        13|          1|
|      14|2015-01-01|  13:14:19|        13|          1|
|      15|2015-01-01|  13:33:00|        13|     

In [0]:
# df = spark.read.format("parquet").load("dbfs:/FileStore/orders")
# df.display()

###Create Permanent Delta table from CSV

In [0]:
location = "/FileStore/order_details.csv"
file_type = "csv"

infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

delta_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(location)


permanent_table_name = "order_details_delta"

delta_df.write.format("delta").saveAsTable(permanent_table_name)

In [0]:
# dbutils.fs.rm('dbfs:/FileStore/orders',recurse=True )
# dbutils.fs.rm('dbfs:/user/hive/warehouse/order_details_delta',recurse=True )
# dbutils.fs.rm('dbfs:/user/hive/warehouse/order_details_delta_',recurse=True )
# dbutils.fs.rm('dbfs:/FileStore/tables/output/Pizza_Data_Analysis',recurse=True )

